In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [ ]:
!pip install bs4
!pip install selenium
!pip install --upgrade pip
!pip install webdriver-manager
!apt update
!apt install chromium-chromedriver

!pip install selenium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb
!pip install tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=002a281ef7f257fe3e9a3961567a4dbaf188ede28b4388a120a8298cdbd1d05c
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/19/5a/a16653bfce685c9832217d377f52065351eeac9862e44e2996cd81f3bb4d/webdriver_manager-4.0.0-py2.py3-none-any.whl.metadata
H

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.chrome.service import Service
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import multiprocessing
from multiprocessing import Process, Manager

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sleepfor1=4  #-----For long task
sleepfor2=3  #-----For short task

def driver_setup():

    options = webdriver.ChromeOptions()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--start-maximized")
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    return driver

In [ ]:
do=driver_setup()
do.quit()

In [ ]:
def get_com_data(browser):
    time.sleep(1)
    browser.get('https://omms.nic.in/')
    time.sleep(1)
    #browser.maximize_window()
    #time.sleep(1)
    input_search = browser.find_element("id",'txtSearchHabs')
    search_button = browser.find_element("xpath","(//button[@type='button'])[3]")
    time.sleep(1)
    input_search.send_keys("jajpur")
    time.sleep(2)
    search_button.click()
    time.sleep(1)
    search_button_next_page = browser.find_element("xpath","//input[@type='button']")
    time.sleep(1)


In [ ]:
def habt(state,dist,block,vi,dataval,err):

    browser = driver_setup()
    get_com_data(browser)

    dropdown1 = Select(browser.find_element("xpath","//select[@id='ddlStates']"))
    dropdown1.select_by_index(state)
    time.sleep(sleepfor2)
    dropdown2 = Select(browser.find_element("xpath","//select[@id='ddlDistricts']"))
    dropdown2.select_by_index(dist)
    time.sleep(sleepfor2)
    dropdown3 = Select(browser.find_element("xpath","//select[@id='ddlSearchBlocks']"))
    dropdown3.select_by_index(block)
    time.sleep(sleepfor2)
    dropdown4 = Select(browser.find_element("xpath","//select[@id='ddlSearchHab']"))
    dropdown4.select_by_index(vi)
    search_button_next_page = browser.find_element("xpath","//input[@type='button']")

    try:

        dropdown4.select_by_index(vi)
        #print(dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
        time.sleep(sleepfor2)

        browser.execute_script("window.scrollTo(0, 0)")

        search_button_next_page.click()#----------------------Click get details button
        state = [['State', 'Dist','Block','Village'],[dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text]]
        statedf = pd.DataFrame(state)
        time.sleep(sleepfor1)#--------------------------------


        expand_rows = browser.find_elements("xpath","//button[@class='btn btn-box-tool']")
        for row in expand_rows:
            row.click()
            time.sleep(sleepfor2)#----------------------------------Sleep after clicking (2 sec)


        div_element = browser.find_element("xpath","//div[@id='divContentTest1']")#-----------------Get table 1
        div_html = div_element.get_attribute("innerHTML")
        soup = BeautifulSoup(div_html, "html.parser")
        table_element = soup.find("table")
        table_data = pd.read_html(str(table_element))
        df = table_data[0]
        tab1=pd.DataFrame([df[1][2:]._append(df[3][2:], ignore_index = True),df[2][2:]._append(df[5][2:], ignore_index = True)])

        #time.sleep(sleepfor)#----------------------------------Sleep not required (maybe)
        table_rows2 = browser.find_elements("xpath","//table[@class='toptable dataTable']/tbody/tr")
        table_data2 = []
        for row2 in table_rows2:
            cells2 = row2.find_elements("xpath",".//td")
            row_data2 = [cell2.text for cell2 in cells2]
            table_data2.append(row_data2)
        df2 = pd.DataFrame(table_data2)
        try:
            df2=df2.iloc[::2]
            df2=df2[df2.columns[:6]]
            df2=df2.sort_index().reset_index(drop=True)
            df2=pd.DataFrame([[None,"Road Name","Road Number","Road Category","Road Length (in Km.)","Link Route / Through Route"]], columns=df2.columns)._append(df2)
            for t in range(len(df2. index)-2):
                df2.loc[t+0.5] = [None,"Road Name","Road Number","Road Category","Road Length (in Km.)","Link Route / Through Route"]
            df2=df2.sort_index().reset_index(drop=True)
            df2=df2[df2.columns[1:]]
            df2temp=df2.iloc[:2]
            for i in range(int(len(df2.index)/2-1)):
                df2temp=pd.concat([df2temp,df2.iloc[2*i+2:2*i+4].sort_index().reset_index(drop=True)],axis=1)
        except:
            df2temp = pd.DataFrame({ 0:[" "] })
            time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)


        time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)
        table_rows3 = browser.find_elements("xpath","//table[@class='toptable']/tbody/tr")
        table_data3 = []
        for row3 in table_rows3:
            cells3 = row3.find_elements("xpath",".//td")
            row_data3 = [cell3.text for cell3 in cells3]
            table_data3.append(row_data3)
        df3 = pd.DataFrame(table_data3)
        try:
            df3=df3.iloc[::2]
            df3=df3[df3.columns[:7]]
            df3=df3.sort_index().reset_index(drop=True)
            df3=pd.DataFrame([[None,"Road/Bridge Name","Package","Phase","Length of Roads(in Kms.) / Bridge (in Mtrs.)","Sanctioned Cost (Rs. in Lacs)","Type Of Proposal"]], columns=df3.columns)._append(df3)
            for t in range(len(df3. index)-2):
                df3.loc[t+0.5] = [None,"Road/Bridge Name","Package","Phase","Length of Roads(in Kms.) / Bridge (in Mtrs.)","Sanctioned Cost (Rs. in Lacs)","Type Of Proposal"]
            df3=df3.sort_index().reset_index(drop=True)
            df3=df3[df3.columns[1:]]
            df3temp=df3.iloc[:2]
            for i in range(int(len(df3.index)/2-1)):
                df3temp=pd.concat([df3temp,df3.iloc[2*i+2:2*i+4].sort_index().reset_index(drop=True)],axis=1)
        except:
            df3temp = pd.DataFrame({ 0:[" "] })
            time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)


        time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)
        expand_rows = browser.find_elements("xpath","//td[@class='collapseRowLnk']")
        for row in expand_rows:
            row.click()
            time.sleep(sleepfor2)#----------------------------------Sleep after selecting (1.5 sec)


        table_rows = browser.find_elements("xpath","//table[@class='nested']/tbody/tr")
        table_data = []
        for row in table_rows:
            cells = row.find_elements("xpath",".//td")
            row_data = [cell.text for cell in cells]
            table_data.append(row_data)
        df = pd.DataFrame(table_data)
        tab2=pd.DataFrame([df[0]._append(df[2], ignore_index = True)._append(df[4], ignore_index = True),df[1]._append(df[3], ignore_index = True)._append(df[5], ignore_index = True)])

        #time.sleep(sleepfor)#----------------------------------Sleep not required (maybe)
        fin=pd.concat([statedf,tab1,df2temp,df3temp, tab2], axis=1)
        #fin.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+dropdown1.first_selected_option.text+'_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
        #print("Success",dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
        browser.quit()
        dataval.append(fin)
        #print("Append success")
    except BaseException as exception:
        state = [['State', 'Dist','Dist Number','Block','Block Number','Village','Village Number','Error type','Error desc'],[dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dist,dropdown3.first_selected_option.text,block,dropdown4.first_selected_option.text,vi,type(exception).__name__,exception]]
        statedf = pd.DataFrame(state)
        #statedf.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+dropdown1.first_selected_option.text+'_error_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
        #print("Fail",dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
        browser.quit()
        err.append(statedf)
        #print("Append f")


In [ ]:
def habt1(statedd,dist,block,vi,dataval,err):
    #print(statedd)
    #print(dist)
    #print(block)
    #print(vi)

    browser = driver_setup()
    get_com_data(browser)
    dropdown1 = Select(browser.find_element("xpath","//select[@id='ddlStates']"))
    dropdown2 = Select(browser.find_element("xpath","//select[@id='ddlDistricts']"))
    dropdown3 = Select(browser.find_element("xpath","//select[@id='ddlSearchBlocks']"))
    dropdown4 = Select(browser.find_element("xpath","//select[@id='ddlSearchHab']"))


    for ggg in range(len(statedd)):
      #print(statedd[ggg]," : ",ggg)
      if ggg==0 or (ggg>=1 and statedd[ggg-1]!=statedd[ggg]):
        dropdown1.select_by_index(statedd[ggg])
        time.sleep(sleepfor2)

      if ggg==0 or (ggg>=1 and dist[ggg-1]!=dist[ggg]):
        dropdown2.select_by_index(dist[ggg])
        time.sleep(sleepfor2)

      if ggg==0 or (ggg>=1 and block[ggg-1]!=block[ggg]):
        dropdown3.select_by_index(block[ggg])
        time.sleep(sleepfor2)

      #browser.execute_script("window.scrollTo(0, 0)")
      dropdown4.select_by_index(vi[ggg])
      search_button_next_page = browser.find_element("xpath","//input[@type='button']")

      try:

          #dropdown4.select_by_index(vi[ggg])
          #print(dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
          time.sleep(sleepfor2)

          browser.execute_script("window.scrollTo(0, 0)")

          search_button_next_page.click()#----------------------Click get details button
          state = [['State', 'Dist','Block','Village'],[dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text]]
          statedf = pd.DataFrame(state)
          time.sleep(sleepfor1)#--------------------------------


          expand_rows = browser.find_elements("xpath","//button[@class='btn btn-box-tool']")
          for row in expand_rows:
              row.click()
              time.sleep(sleepfor2)#----------------------------------Sleep after clicking (2 sec)


          div_element = browser.find_element("xpath","//div[@id='divContentTest1']")#-----------------Get table 1
          div_html = div_element.get_attribute("innerHTML")
          soup = BeautifulSoup(div_html, "html.parser")
          table_element = soup.find("table")
          table_data = pd.read_html(str(table_element))
          df = table_data[0]
          tab1=pd.DataFrame([df[1][2:]._append(df[3][2:], ignore_index = True),df[2][2:]._append(df[5][2:], ignore_index = True)])

          #time.sleep(sleepfor)#----------------------------------Sleep not required (maybe)
          table_rows2 = browser.find_elements("xpath","//table[@class='toptable dataTable']/tbody/tr")
          table_data2 = []
          for row2 in table_rows2:
              cells2 = row2.find_elements("xpath",".//td")
              row_data2 = [cell2.text for cell2 in cells2]
              table_data2.append(row_data2)
          df2 = pd.DataFrame(table_data2)
          try:
              df2=df2.iloc[::2]
              df2=df2[df2.columns[:6]]
              df2=df2.sort_index().reset_index(drop=True)
              df2=pd.DataFrame([[None,"Road Name","Road Number","Road Category","Road Length (in Km.)","Link Route / Through Route"]], columns=df2.columns)._append(df2)
              for t in range(len(df2. index)-2):
                  df2.loc[t+0.5] = [None,"Road Name","Road Number","Road Category","Road Length (in Km.)","Link Route / Through Route"]
              df2=df2.sort_index().reset_index(drop=True)
              df2=df2[df2.columns[1:]]
              df2temp=df2.iloc[:2]
              for i in range(int(len(df2.index)/2-1)):
                  df2temp=pd.concat([df2temp,df2.iloc[2*i+2:2*i+4].sort_index().reset_index(drop=True)],axis=1)
          except:
              df2temp = pd.DataFrame({ 0:[" "] })
              time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)


          time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)
          table_rows3 = browser.find_elements("xpath","//table[@class='toptable']/tbody/tr")
          table_data3 = []
          for row3 in table_rows3:
              cells3 = row3.find_elements("xpath",".//td")
              row_data3 = [cell3.text for cell3 in cells3]
              table_data3.append(row_data3)
          df3 = pd.DataFrame(table_data3)
          try:
              df3=df3.iloc[::2]
              df3=df3[df3.columns[:7]]
              df3=df3.sort_index().reset_index(drop=True)
              df3=pd.DataFrame([[None,"Road/Bridge Name","Package","Phase","Length of Roads(in Kms.) / Bridge (in Mtrs.)","Sanctioned Cost (Rs. in Lacs)","Type Of Proposal"]], columns=df3.columns)._append(df3)
              for t in range(len(df3. index)-2):
                  df3.loc[t+0.5] = [None,"Road/Bridge Name","Package","Phase","Length of Roads(in Kms.) / Bridge (in Mtrs.)","Sanctioned Cost (Rs. in Lacs)","Type Of Proposal"]
              df3=df3.sort_index().reset_index(drop=True)
              df3=df3[df3.columns[1:]]
              df3temp=df3.iloc[:2]
              for i in range(int(len(df3.index)/2-1)):
                  df3temp=pd.concat([df3temp,df3.iloc[2*i+2:2*i+4].sort_index().reset_index(drop=True)],axis=1)
          except:
              df3temp = pd.DataFrame({ 0:[" "] })
              time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)


          time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)
          expand_rows = browser.find_elements("xpath","//td[@class='collapseRowLnk']")
          for row in expand_rows:
              row.click()
              time.sleep(sleepfor2)#----------------------------------Sleep after selecting (1.5 sec)


          table_rows = browser.find_elements("xpath","//table[@class='nested']/tbody/tr")
          table_data = []
          for row in table_rows:
              cells = row.find_elements("xpath",".//td")
              row_data = [cell.text for cell in cells]
              table_data.append(row_data)
          df = pd.DataFrame(table_data)
          tab2=pd.DataFrame([df[0]._append(df[2], ignore_index = True)._append(df[4], ignore_index = True),df[1]._append(df[3], ignore_index = True)._append(df[5], ignore_index = True)])

          #time.sleep(sleepfor)#----------------------------------Sleep not required (maybe)
          fin=pd.concat([statedf,tab1,df2temp,df3temp, tab2], axis=1)
          #fin.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+dropdown1.first_selected_option.text+'_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
          #print("Success",dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
          #browser.quit()
          dataval.append(fin)
          #print("Append success")
      except BaseException as exception:
          state = [['State', 'Dist','Dist Number','Block','Block Number','Village','Village Number','Error type','Error desc'],[dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dist[ggg],dropdown3.first_selected_option.text,block[ggg],dropdown4.first_selected_option.text,vi[ggg],type(exception).__name__,exception]]
          statedf = pd.DataFrame(state)
          #statedf.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+dropdown1.first_selected_option.text+'_error_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
          #print("Fail",dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
          #browser.quit()
          err.append(statedf)
          #print("Append f")
    browser.quit()

In [ ]:
def habt2(statedd,dist,block,vi,dataval,err,browser):
    #print(statedd)
    #print(dist)
    #print(block)
    #print(vi)

    #browser = driver_setup()
    #get_com_data(browser)
    dropdown1 = Select(browser.find_element("xpath","//select[@id='ddlStates']"))
    dropdown2 = Select(browser.find_element("xpath","//select[@id='ddlDistricts']"))
    dropdown3 = Select(browser.find_element("xpath","//select[@id='ddlSearchBlocks']"))
    dropdown4 = Select(browser.find_element("xpath","//select[@id='ddlSearchHab']"))


    for ggg in range(len(statedd)):
      #print(statedd[ggg]," : ",ggg)
      if ggg==0 or (ggg>=1 and statedd[ggg-1]!=statedd[ggg]):
        dropdown1.select_by_index(statedd[ggg])
        time.sleep(sleepfor2)

      if ggg==0 or (ggg>=1 and dist[ggg-1]!=dist[ggg]):
        dropdown2.select_by_index(dist[ggg])
        time.sleep(sleepfor2)

      if ggg==0 or (ggg>=1 and block[ggg-1]!=block[ggg]):
        dropdown3.select_by_index(block[ggg])
        time.sleep(sleepfor2)

      #browser.execute_script("window.scrollTo(0, 0)")
      dropdown4.select_by_index(vi[ggg])
      search_button_next_page = browser.find_element("xpath","//input[@type='button']")

      try:

          #dropdown4.select_by_index(vi[ggg])
          #print(dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
          time.sleep(sleepfor2)

          browser.execute_script("window.scrollTo(0, 0)")

          search_button_next_page.click()#----------------------Click get details button
          state = [['State', 'Dist','Block','Village'],[dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text]]
          statedf = pd.DataFrame(state)
          time.sleep(sleepfor1)#--------------------------------


          expand_rows = browser.find_elements("xpath","//button[@class='btn btn-box-tool']")
          for row in expand_rows:
              row.click()
              time.sleep(sleepfor2)#----------------------------------Sleep after clicking (2 sec)


          div_element = browser.find_element("xpath","//div[@id='divContentTest1']")#-----------------Get table 1
          div_html = div_element.get_attribute("innerHTML")
          soup = BeautifulSoup(div_html, "html.parser")
          table_element = soup.find("table")
          table_data = pd.read_html(str(table_element))
          df = table_data[0]
          tab1=pd.DataFrame([df[1][2:]._append(df[3][2:], ignore_index = True),df[2][2:]._append(df[5][2:], ignore_index = True)])

          #time.sleep(sleepfor)#----------------------------------Sleep not required (maybe)
          table_rows2 = browser.find_elements("xpath","//table[@class='toptable dataTable']/tbody/tr")
          table_data2 = []
          for row2 in table_rows2:
              cells2 = row2.find_elements("xpath",".//td")
              row_data2 = [cell2.text for cell2 in cells2]
              table_data2.append(row_data2)
          df2 = pd.DataFrame(table_data2)
          try:
              df2=df2.iloc[::2]
              df2=df2[df2.columns[:6]]
              df2=df2.sort_index().reset_index(drop=True)
              df2=pd.DataFrame([[None,"Road Name","Road Number","Road Category","Road Length (in Km.)","Link Route / Through Route"]], columns=df2.columns)._append(df2)
              for t in range(len(df2. index)-2):
                  df2.loc[t+0.5] = [None,"Road Name","Road Number","Road Category","Road Length (in Km.)","Link Route / Through Route"]
              df2=df2.sort_index().reset_index(drop=True)
              df2=df2[df2.columns[1:]]
              df2temp=df2.iloc[:2]
              for i in range(int(len(df2.index)/2-1)):
                  df2temp=pd.concat([df2temp,df2.iloc[2*i+2:2*i+4].sort_index().reset_index(drop=True)],axis=1)
          except:
              df2temp = pd.DataFrame({ 0:[" "] })
              time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)


          time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)
          table_rows3 = browser.find_elements("xpath","//table[@class='toptable']/tbody/tr")
          table_data3 = []
          for row3 in table_rows3:
              cells3 = row3.find_elements("xpath",".//td")
              row_data3 = [cell3.text for cell3 in cells3]
              table_data3.append(row_data3)
          df3 = pd.DataFrame(table_data3)
          try:
              df3=df3.iloc[::2]
              df3=df3[df3.columns[:7]]
              df3=df3.sort_index().reset_index(drop=True)
              df3=pd.DataFrame([[None,"Road/Bridge Name","Package","Phase","Length of Roads(in Kms.) / Bridge (in Mtrs.)","Sanctioned Cost (Rs. in Lacs)","Type Of Proposal"]], columns=df3.columns)._append(df3)
              for t in range(len(df3. index)-2):
                  df3.loc[t+0.5] = [None,"Road/Bridge Name","Package","Phase","Length of Roads(in Kms.) / Bridge (in Mtrs.)","Sanctioned Cost (Rs. in Lacs)","Type Of Proposal"]
              df3=df3.sort_index().reset_index(drop=True)
              df3=df3[df3.columns[1:]]
              df3temp=df3.iloc[:2]
              for i in range(int(len(df3.index)/2-1)):
                  df3temp=pd.concat([df3temp,df3.iloc[2*i+2:2*i+4].sort_index().reset_index(drop=True)],axis=1)
          except:
              df3temp = pd.DataFrame({ 0:[" "] })
              time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)


          time.sleep(sleepfor2)#----------------------------------Sleep not required (maybe)
          expand_rows = browser.find_elements("xpath","//td[@class='collapseRowLnk']")
          for row in expand_rows:
              row.click()
              time.sleep(sleepfor2)#----------------------------------Sleep after selecting (1.5 sec)


          table_rows = browser.find_elements("xpath","//table[@class='nested']/tbody/tr")
          table_data = []
          for row in table_rows:
              cells = row.find_elements("xpath",".//td")
              row_data = [cell.text for cell in cells]
              table_data.append(row_data)
          df = pd.DataFrame(table_data)
          tab2=pd.DataFrame([df[0]._append(df[2], ignore_index = True)._append(df[4], ignore_index = True),df[1]._append(df[3], ignore_index = True)._append(df[5], ignore_index = True)])

          #time.sleep(sleepfor)#----------------------------------Sleep not required (maybe)
          fin=pd.concat([statedf,tab1,df2temp,df3temp, tab2], axis=1)
          #fin.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+dropdown1.first_selected_option.text+'_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
          #print("Success",dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
          #browser.quit()
          dataval.append(fin)
          #print("Append success")
      except BaseException as exception:
          state = [['State', 'Dist','Dist Number','Block','Block Number','Village','Village Number','Error type','Error desc'],[dropdown1.first_selected_option.text,dropdown2.first_selected_option.text,dist[ggg],dropdown3.first_selected_option.text,block[ggg],dropdown4.first_selected_option.text,vi[ggg],type(exception).__name__,exception]]
          statedf = pd.DataFrame(state)
          #statedf.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+dropdown1.first_selected_option.text+'_error_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
          #print("Fail",dropdown2.first_selected_option.text,dropdown3.first_selected_option.text,dropdown4.first_selected_option.text)
          #browser.quit()
          err.append(statedf)
          #print("Append f")
    #browser.quit()

In [ ]:
partnumber=32#--------Part 3, 12, 23, 32
statenumber=35
statename="Uttar Pradesh"
threads=20

ss = pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_manual_part_'+str(partnumber)+'.csv')
ssdone=pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_done_data_part_'+str(partnumber)+'.csv')
for i in tqdm(range(len(ssdone),len(ssdone)+50,threads)):
  prop=[]
  with Manager() as ma:
    res=ma.list()
    ert=ma.list()
    for ll in range(threads):
      print(ss.iloc[[i+ll]])
    print("------------------------------------------------------------------")

  0%|          | 0/3 [00:00<?, ?it/s]


IndexError: ignored

In [ ]:
ttt=[]
ghj=[]
#habt(35,1,1,1,ttt,ghj)
print(ttt)
ttt=[]
ghj=[]
#habt1([35,35],[3,3],[1,1],[158,159],ttt,ghj)
hj=driver_setup()
get_com_data(hj)
habt2([35,35],[17,17],[6,7],[350,0],ttt,ghj,hj)
print(ttt)
print(ghj)

[]
[              0       1        2                                                3                 0                     1               2              3                    4            5               6                       7                 8                     9               10           11                      12             13                     14               15          16           17    18                  19               20                  21           22               23  24  25  26  27                                   1            2                          3                     4                           5    0                                    0                        1                   2          3                      4     5          6          7     8 
0          State    Dist    Block                                          Village  Total Population  Educational Services  Primary School  Middle School  Intermediate School  ITI College  Degree College 

In [ ]:
hj.quit()

In [ ]:
partnumber=32#----------Part 3, 12, 23, 32
statenumber=35
statename="Uttar Pradesh"
threads=1
subs=1


brs=[driver_setup() for i in range(threads)]
po=[]
for i in range(threads):
  p=Process(target=get_com_data,args=[brs[i]])
  p.start()
  po.append(p)
  #get_com_data(brs[i])
for i in po:
  i.join()



ss = pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_manual_part_'+str(partnumber)+'.csv')
ssdone=pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_done_data_part_'+str(partnumber)+'.csv')
for i in tqdm(range(len(ssdone),len(ss),threads*subs)):
  prop=[]
  with Manager() as ma:
    res=ma.list()
    ert=ma.list()
    for ll in range(threads*subs):
      print(ss.iloc[[i+ll]])
    for j in range(0,threads*subs,subs):
      p=Process(target=habt2,args=[[statenumber for k in range(subs)],[int(ss.iloc[[i+j+k]]["dist"]) for k in range(subs)],[int(ss.iloc[[i+j+k]]["block"]) for k in range(subs)],[int(ss.iloc[[i+j+k]]["village"]) for k in range(subs)],res,ert,brs[int(j/subs)]])
      #print(i+j)
      p.start()
      prop.append(p)
    for t in prop:
      #print(i)
      t.join()
    print(len(res),len(ert))
    for resul in res:
      print(resul)
      resul.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
    for ers in ert:
      print(ers)
      ers.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_error_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
    for ll in range(threads*subs):
      #print(ss.iloc[[i+ll]])
      ss.iloc[[i+ll]].to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_done_data_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)


  0%|          | 0/12 [00:00<?, ?it/s]

      dist  block  village
6330    61      4      141


  8%|▊         | 1/12 [00:46<08:29, 46.29s/it]

1 0
              0           1       2                                  3                 0                     1               2              3                    4            5               6                       7                 8                     9               10           11                      12             13                     14               15          16           17    18                  19               20                  21           22               23  24  25  26  27                          1            2                          3                     4                           5                          1        2          3                                             4                              5                 6                           0                        1           2        3                   4                   5                    6          7                      8     9                   10               11                        12

 17%|█▋        | 2/12 [01:37<08:11, 49.18s/it]

...

1 0
              0           1       2                                 3                 0                     1               2              3                    4            5               6                       7                 8                     9               10           11                      12             13                     14               15          16           17    18                  19               20                  21           22               23  24  25  26  27                                1            2                          3                     4                           5    0                                 0                        1                2          3                      4     5          6          7     8 
0          State        Dist   Block                           Village  Total Population  Educational Services  Primary School  Middle School  Intermediate School  ITI College  Degree College  Communication Services  Telegrap

 92%|█████████▏| 11/12 [08:36<00:46, 46.52s/it]

1 0
              0           1       2                               3                 0                     1               2              3                    4            5               6                       7                 8                     9               10           11                      12             13                     14               15          16           17    18                  19               20                  21           22               23  24  25  26  27                    1            2                          3                     4                           5                           1        2          3                                             4                              5                 6                     0                        1        2        3                   4                   5                    6          7                      8     9              10               11                        12               13  \
0

100%|██████████| 12/12 [09:29<00:00, 47.49s/it]

1 0
              0           1       2                                  3                 0                     1               2              3                    4            5               6                       7                 8                     9               10           11                      12             13                     14               15          16           17    18                  19               20                  21           22               23  24  25  26  27                                  1            2                          3                     4                           5                                    1        2          3                                             4                              5                 6                                                  1        2          3                                             4                              5                 6                                   0   \
0          Sta

In [ ]:
partnumber=3#----------Part 3
statenumber=35
statename="Uttar Pradesh"
threads=20

ss = pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_manual_part_'+str(partnumber)+'.csv')
ssdone=pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_done_data_part_'+str(partnumber)+'.csv')
for i in tqdm(range(len(ssdone),len(ss),threads)):
  prop=[]
  with Manager() as ma:
    res=ma.list()
    ert=ma.list()
    for ll in range(threads):
      print(ss.iloc[[i+ll]])
    for j in range(threads):
      p=Process(target=habt,args=[statenumber,int(ss.iloc[[i+j]]["dist"]),int(ss.iloc[[i+j]]["block"]),int(ss.iloc[[i+j]]["village"]),res,ert])
      #print(i+j)
      p.start()
      prop.append(p)
    for t in prop:
      #print(i)
      t.join()
    print(len(res),len(ert))
    for resul in res:
      print(resul)
      resul.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
    for ers in ert:
      print(ers)
      ers.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_error_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)
    for ll in range(threads):
      #print(ss.iloc[[i+ll]])
      ss.iloc[[i+ll]].to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_done_data_part_'+str(partnumber)+'.csv', mode='a', index=False, header=False)


  0%|          | 0/280 [00:00<?, ?it/s]

     dist  block  village
760     4      6        9
     dist  block  village
761     4      6       10
     dist  block  village
762     4      6       11
     dist  block  village
763     4      6       12
     dist  block  village
764     4      6       13
     dist  block  village
765     4      6       14
     dist  block  village
766     4      6       15
     dist  block  village
767     4      6       16
     dist  block  village
768     4      6       17
     dist  block  village
769     4      6       18
     dist  block  village
770     4      6       19
     dist  block  village
771     4      6       20
     dist  block  village
772     4      6       21
     dist  block  village
773     4      6       22
     dist  block  village
774     4      6       23
     dist  block  village
775     4      6       24
     dist  block  village
776     4      6       25
     dist  block  village
777     4      6       26
     dist  block  village
778     4      6       27
     dist  b

  0%|          | 1/280 [03:47<17:38:06, 227.55s/it]

               0       1            2          3             4                            5               6           7           8
0          State    Dist  Dist Number      Block  Block Number                      Village  Village Number  Error type  Error desc
1  Uttar Pradesh  Amethi            4  Gauriganj             6  Babu Pur Village : BABU PUR              14    KeyError           0
               0       1            2          3             4                                             5               6           7           8
0          State    Dist  Dist Number      Block  Block Number                                       Village  Village Number  Error type  Error desc
1  Uttar Pradesh  Amethi            4  Gauriganj             6  Bali Pur Khurdwan Village : BaliPur Khurdwan              20    KeyError           0
     dist  block  village
780     4      6       29
     dist  block  village
781     4      6       30
     dist  block  village
782     4      6       31
 

  1%|          | 2/280 [08:01<18:47:08, 243.27s/it]

...

  9%|▉         | 25/280 [3:48:16<56:48:53, 802.09s/it]

      dist  block  village
1260     4      6      509
      dist  block  village
1261     4      6      510
      dist  block  village
1262     4      6      511
      dist  block  village
1263     4      6      512
      dist  block  village
1264     4      6      513
      dist  block  village
1265     4      6      514
      dist  block  village
1266     4      6      515
      dist  block  village
1267     4      6      516
      dist  block  village
1268     4      6      517
      dist  block  village
1269     4      6      518
      dist  block  village
1270     4      6      519
      dist  block  village
1271     4      6      520
      dist  block  village
1272     4      6      521
      dist  block  village
1273     4      6      522
      dist  block  village
1274     4      6      523
      dist  block  village
1275     4      6      524
      dist  block  village
1276     4      6      525
      dist  block  village
1277     4      6      526
      dist  block  village
1

  9%|▉         | 25/280 [4:02:12<41:10:27, 581.28s/it]

18 2
              0       1          2                                          3                 0                     1               2              3                    4            5               6                       7                 8                     9               10           11                      12             13                     14               15          16           17    18                  19               20                  21           22               23  24  25  26  27                                    1            2                          3                     4                           5    0                                     0                        1              2          3                      4     5          6                   7     8 
0          State    Dist      Block                                    Village  Total Population  Educational Services  Primary School  Middle School  Intermediate School  ITI College  Degree College  

OSError: ignored

In [ ]:
ss = pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+str(statename)+'_manual_part_'+str(partnumber)+'.csv')
ss

,dist,block,village
0,60,12,563
1,60,12,564
2,60,12,565
3,60,12,566
4,60,12,567
...,...,...,...
6337,61,4,148
6338,61,4,149
6339,61,4,150
6340,61,4,151


In [ ]:
partnumber=23#----------Part 3, 12, 23, 32, 1, 11, 16, 28, 38, 2, 15, 20, 26, 34
statenumber=35
statename="Uttar Pradesh"
kj=pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_error_part_'+str(partnumber)+'.csv')
asw=kj.iloc[::2]
duplicate = asw[asw.duplicated()]
duplicate

,State,Dist,Dist Number,Block,Block Number,Village,Village Number,Error type,Error desc


In [ ]:

part=[3, 12, 23, 32, 1, 11, 16, 28, 38, 2, 15, 20, 26, 34]#----------Part 3, 12, 23, 32, 1, 11, 16, 28, 38, 2, 15, 20, 26, 34
statenumber=35
statename="Uttar Pradesh"

col_names = ["col"+str(i) for i in range(10)]
df_append = pd.DataFrame()
#append all files together
for partnumber in part:
            df_temp = pd.read_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_error_part_'+str(partnumber)+'.csv', names=col_names, low_memory=False)
            df_append = df_append.append(df_temp, ignore_index=True)
asw=df_append.iloc[1::2]
duplicate = asw[asw.duplicated()]
duplicate

<ipython-input-110-55601d0e4f71>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
<ipython-input-110-55601d0e4f71>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
<ipython-input-110-55601d0e4f71>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
<ipython-input-110-55601d0e4f71>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
<ipython-input-110-55601d0e4f71>:10: FutureWarning: The frame.append method is depre

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9


In [ ]:
lo=[[i-1,i] for i in duplicate[duplicate == True ].index.tolist()]
fl = [num for row in lo for num in row]
df_append=df_append.drop(index=df_append.iloc[fl].index)
asw=df_append.iloc[1::2]
duplicate = asw[asw.duplicated()]
duplicate

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205,col206,col207,col208,col209,col210,col211,col212,col213,col214,col215,col216,col217,col218,col219,col220,col221,col222,col223,col224,col225,col226,col227,col228,col229,col230,col231,col232,col233,col234,col235,col236,col237,col238,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255,col256,col257,col258,col259,col260,col261,col262,col263,col264,col265,col266,col267,col268,col269,col270,col271,col272,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282,col283,col284,col285,col286,col287,col288,col289,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299


In [ ]:
df_append.to_csv('/content/drive/MyDrive/My Folder/'+str(statenumber)+'/'+statename+'_final_error_part_'+str(1)+'.csv', mode='a', index=False, header=False)

In [ ]:
df_append

,State,Dist,Dist Number,Block,Block Number,Village,Village Number,Error type,Error desc
0,Uttar Pradesh,Amethi,4,Bhadar,4,ACKWA PUR Village : Bhadar,0,KeyError,0
1,State,Dist,Dist Number,Block,Block Number,Village,Village Number,Error type,Error desc
2,Uttar Pradesh,Amethi,4,Bhadar,4,ARKI TIWARI KA PURVA Village : Piper pur,8,KeyError,0
3,State,Dist,Dist Number,Block,Block Number,Village,Village Number,Error type,Error desc
4,Uttar Pradesh,Amethi,4,Bhadar,4,AMMU PUR Village : Bahadur pur,6,KeyError,0
...,...,...,...,...,...,...,...,...,...
31329,Uttar Pradesh,Sambhal,66,Rajpura,7,Madhaiya Singhola Chet Village : Madhaiya Sing...,68,KeyError,0
31330,State,Dist,Dist Number,Block,Block Number,Village,Village Number,Error type,Error desc
31331,Uttar Pradesh,Sambhal,66,Rajpura,7,Paratpur Village : Paratpur,90,KeyError,0
31332,State,Dist,Dist Number,Block,Block Number,Village,Village Number,Error type,Error desc
